# Churn automl 
with batch transform and clarify

Steve Goodman
Feb 2021


This is a MVP to build a model quickly, using simple dataset that simulates telco churn. Note the other notebook in this repo which comes from AWS examples is more comprehensive and well explained but missing a few key points:
1. We normally have a row identifier such as customer id when scoring the data
2. Explainability is important. 

We are going to 
1. Download the dataset, and make some small data prep changes to it
2. Run Autopilot
3. Evaluate the best model using the auc metric
4. Use Clarify to Explain what features are most important.
5. Run a batch transform job on the test data set to generate predictions. 
6. Evaluate predictions 


todos - 
* align the user ids and figure if column headers are allowed in the test data or if we have to add them later
* also whether y column can/can't be in test data. 
* clean the directories so all in the same project, and in sub directories inputs/training/test, inference/output.


## Resources

Links to posts about Autopilot, Clarify. The first link is a fuller example of this churn example with more detailed explainations. (Its missing a couple of issues that I think are important however)

https://github.com/aws/amazon-sagemaker-examples/blob/master/autopilot/autopilot_customer_churn_high_level_with_evaluation.ipynb


https://aws.amazon.com/blogs/machine-learning/explaining-amazon-sagemaker-autopilot-models-with-shap/

https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf

https://aws.amazon.com/blogs/machine-learning/creating-high-quality-machine-learning-models-for-financial-services-using-amazon-sagemaker-autopilot/

https://aws.amazon.com/blogs/aws/new-amazon-sagemaker-clarify-detects-bias-and-increases-the-transparency-of-machine-learning-models/


In [1]:
import pandas as pd
import numpy as np
import os
from time import gmtime, strftime, sleep

from sklearn.model_selection import train_test_split

import boto3
import sagemaker  
from sagemaker.automl.automl import AutoML


In [2]:
sm = boto3.client('sagemaker')
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()


In [40]:
# prefix is where the project will be located under the sagemaker studio bucket
# probably more sensible to use something like CLIENT_NAME/PROJECT_NAME or similar
# to separate it from your colleagues


prefix = 'sagemaker/' + auto_ml_job_name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
auto_ml_job_name = 'automl-churn' + timestamp_suffix # This will be the s3 bucket name so dashes are ok but underscores are not 


model_name = 'automl-churn-model-' + timestamp_suffix
transform_job_name = "automl-churn-model-transform-" + timestamp_suffix

test_data_s3_path = f's3://{bucket}/{prefix}/test/'
s3_transform_output_path = f's3://{bucket}/{prefix}/inference-results/'


In [50]:
timestamp_suffix

'17-14-42-12'

# Step 1 - Get datasets 
Not a modelling task - this should ordinarily be done in a separate preprocessing step to e.g. pull data from the Lake or Database.
But some key things of note
1. There is no customer id in the dataset, so we're going to use telephone number (which is unique) as a proxy.
2. The training dataset needs a Target column (churn flag) but ordinarily, this will be abscent from the scoring data- so will drop it from our 'test' dataset. The test set in this case is used to demonstrate how to simulate a scoring job.
3. The training dataset should have column headers in the first row, however, the scoring dataset does not (just how batch transform works it would appear)


In [27]:
os.chdir('/root/churn')

In [28]:
!pwd

/root/churn


In [29]:
#!apt install unzip
#!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
#!unzip -o DKD2e_data_sets.zip


In [30]:
churn = pd.read_csv('./raw/churn.txt')

In [31]:
churn.shape

(3333, 21)

In [32]:
#There is no explict customer id- so lets use tel number as a proxy
churn.rename(columns={'Phone': 'cust_id'}, inplace = True)

In [33]:
# move cust_id to the first column - not compusory but will be convenient later
cols = list(churn)
cols.insert(0, cols.pop(cols.index('cust_id')))
churn = churn.loc[:, cols]

In [34]:
# the target column is not well formed - lets turn it to an indicator flag instead

churn['churn_flag'] = np.where(churn['Churn?']=='False.', 0, 1)
churn.drop('Churn?', axis='columns', inplace=True)
churn['churn_flag'].value_counts()


0    2850
1     483
Name: churn_flag, dtype: int64

In [35]:
churn.head()

,cust_id,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,churn_flag
0,382-4657,KS,128,415,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,371-7191,OH,107,415,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,358-1921,NJ,137,415,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,375-9999,OH,84,408,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,330-6626,OK,75,415,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [36]:
#Split the data into 2 files - 1 for training, the 'test' data will be for demo-ing batch inference
y = churn['churn_flag']
X = churn.drop('churn_flag', axis='columns')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [37]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2233, 20), (1100, 20), (2233,), (1100,))

In [38]:
train_file = 'train/train_data.csv';
test_file = 'test/test_data.csv';

In [44]:
#for the training data we drop the cust_id as its not a feature column

# for the test data ..
# we retain custid - we will need at at inference time to match score to customer
# drop the header row - ( batch transform expects no header)
# drop the churn flag - we assume the ground truth  is not known during inference

#upload both datasets to s3

training_data = pd.DataFrame(X_train)
training_data.drop('cust_id', axis='columns',inplace=True)
training_data['churn_flag'] = list(y_train)
training_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)


# going to drop the header for inference purposes. 
test_data = pd.DataFrame(X_test)
test_data.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)


Train data uploaded to: s3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/train/train_data.csv
Test data uploaded to: s3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/test/test_data.csv


In [45]:
# so the uploaded training data looks like this:

trd = pd.read_csv('s3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/train/train_data.csv')
trd.head()
    

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,churn_flag
0,MA,119,408,yes,yes,16,147.2,103,25.02,160.1,96,13.61,184.0,120,8.28,7.7,2,2.08,0,1
1,VA,136,415,no,yes,35,205.5,86,34.94,298.5,119,25.37,214.2,104,9.64,6.9,4,1.86,1,0
2,MO,147,415,yes,no,0,157.0,79,26.69,103.1,94,8.76,211.8,96,9.53,7.1,6,1.92,0,0
3,MI,61,415,no,yes,33,270.7,53,46.02,200.7,116,17.06,201.7,102,9.08,10.9,3,2.94,3,0
4,DC,68,415,yes,yes,39,142.0,140,24.14,241.6,89,20.54,302.0,72,13.59,11.3,5,3.05,1,0


In [46]:
# and the uploaded test data looks like this:

testd = pd.read_csv('s3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/test/test_data.csv')
testd.head()

,335-4719,LA,117,408,no,no.1,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1
0,379-3012,CO,83,510,no,no,0,132.4,120,22.51,121.6,101,10.34,197.7,84,8.90,8.6,2,2.32,1
1,406-5023,DC,93,408,no,no,0,164.5,95,27.97,230.9,87,19.63,149.9,91,6.75,9.9,3,2.67,4
2,333-9511,DC,141,415,no,yes,37,185.4,87,31.52,178.5,128,15.17,218.3,107,9.82,8.0,3,2.16,4
3,405-7204,ID,82,510,no,yes,34,232.6,121,39.54,153.2,115,13.02,286.7,77,12.90,4.7,3,1.27,3
4,399-9239,ID,169,415,no,no,0,235.7,79,40.07,136.9,85,11.64,220.9,97,9.94,13.3,10,3.59,1


In [47]:
training_data.shape, test_data.shape

((2233, 20), (1100, 20))

# Step 2 - Training Job
We will ask Autopilot to generate a limited amount of models (if you stick with the defaults the run could take several hrs). We will then retrieve the best one based on the evaluation metric - AUC.
Note you can also run the job through the UI from the launcher

In [48]:
automl = AutoML(role=role,
                target_attribute_name="churn_flag",
                base_job_name=auto_ml_job_name,
                sagemaker_session=session,
                problem_type='BinaryClassification',
                job_objective={'MetricName': 'AUC'},
                max_candidates=5)                

In [49]:
automl.fit(train_file, job_name=auto_ml_job_name, wait=False, logs=False)
describe_response = automl.describe_auto_ml_job()
print (describe_response)
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = automl.describe_auto_ml_job()
    job_run_status = describe_response['AutoMLJobStatus']
    print (job_run_status)
    sleep(30)
print ('completed')

{'AutoMLJobName': 'automl-churn17-14-42-12', 'AutoMLJobArn': 'arn:aws:sagemaker:eu-west-1:114936231890:automl-job/automl-churn17-14-42-12', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-eu-west-1-114936231890/auto-ml-input-data/train_data.csv'}}, 'TargetAttributeName': 'churn_flag'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-eu-west-1-114936231890/'}, 'RoleArn': 'arn:aws:iam::114936231890:role/service-role/AmazonSageMaker-ExecutionRole-20210125T152753', 'AutoMLJobObjective': {'MetricName': 'AUC'}, 'ProblemType': 'BinaryClassification', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 5}, 'SecurityConfig': {'EnableInterContainerTrafficEncryption': False}}, 'CreationTime': datetime.datetime(2021, 2, 17, 14, 44, 26, 666000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2021, 2, 17, 14, 44, 26, 666000, tzinfo=tzlocal()), 'AutoMLJobStatus': 'InProgress', 'AutoMLJobSecondaryStatus': 'Starting', 'Gener

In [51]:
best_candidate = automl.describe_auto_ml_job()['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))


CandidateName: tuning-job-1-c0d70d5fb80d43d7a2-002-bd2c97a7
FinalAutoMLJobObjectiveMetricName: validation:auc
FinalAutoMLJobObjectiveMetricValue: 0.8993300199508667



----
*Note* we are taking the best candidate forward for production, but we could equally choose one of the other models generated if we wish. Remember the CandidateName - this is the model name we will use from now on.

# Step 3 -  Explainability
Run AWS clarify to get global shap values



In [5]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.c4.xlarge',
                                                      sagemaker_session=session)

#Pass it the best model name here....
model_config =  clarify.ModelConfig(model_name='tuning-job-1-8722c107ed7946f791-003-a55c8eb6',
                                   instance_type='ml.c5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv')

In [17]:
test_file = 'test/test_data.csv';
test_data = pd.read_csv(test_file)
                            

In [37]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs')

In [35]:
test_data.dtypes

cust_id            object
State              object
Account Length      int64
Area Code           int64
Int'l Plan         object
VMail Plan         object
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls          int64
Intl Charge       float64
CustServ Calls      int64
dtype: object

In [36]:
# problem : the clarify won't run this as-is because it doesn't like Ints


# See for issue and workarounds
# https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable


#Solution
test_data = test_data.astype({'Account Length':'float64', 'Area Code':'float64', 
                             'VMail Message':'float64', 'Day Calls':'float64',
                             'Eve Calls':'float64', 'Night Calls':'float64',
                             'Intl Calls':'float64', 'CustServ Calls':'float64'})



In [28]:
train_uri = f's3://{bucket}/{prefix}/train/train_data.csv'
explainability_output_path = f's3://{bucket}/{prefix}/exaplainability/'



In [29]:
train_uri

's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churnv4/train/train_data.csv'

In [40]:
explainability_output_path = f's3://{bucket}/{prefix}/clarify-explainability'
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='churn_flag',
                                headers=training_data.columns.to_list(),
                                dataset_type='text/csv')

In [31]:
# shap_config = clarify.SHAPConfig(baseline=[test_features.iloc[0].values.tolist()],
#                                  num_samples=15,
#                                  agg_method='mean_abs')

In [ ]:
"ExperimentConfig": { 
      "ExperimentName": "string",
      "TrialComponentDisplayName": "string",
      "TrialName": "string"
   },

In [41]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)


Job Name:  Clarify-Explainability-2021-02-09-15-40-02-126
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churnv4/train/train_data.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-114936231890/Clarify-Explainability-2021-02-09-15-40-02-126/input/analysis_config/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churnv4/clarify-explainability', 'LocalPath': '/opt/ml/processing/output', 'S3Uploa

In [3]:
!conda install -c conda-forge -y shap

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - shap


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py37h89c1867_0         3.0 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    shap-0.37.0                |   py37h10a2094_0         510 KB  conda-forge
    slicer-0.0.7               |     pyhd8ed1ab_0          16 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.7-1_cp37

In [15]:
import shap 
from  io import StringIO

In [18]:
training_data.columns

Index(['cust_id', 'State', 'Account Length', 'Area Code', 'Int'l Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'churn_flag'],
      dtype='object')

In [19]:
#This is the right template but not working - training_data and shap_values matrix are different shapes
output = sagemaker.s3.S3Downloader.read_file("""s3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churnv4/clarify-explainability/explanations_shap/out.csv""")
local_shap_values = pd.read_csv(StringIO(output), sep=',', header=None)

#keep only columns where target is 1, not 0
for col in local_shap_values.columns:

features = training_data.drop(columns='churn_flag')
shap.summary_plot(shap_values = local_shap_values.to_numpy(), features=features.to_numpy(), 
                 feature_names=features.columns)

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

In [24]:
local_shap_values.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,cust_id_label0,State_label0,Account Length_label0,Area Code_label0,Int'l Plan_label0,VMail Plan_label0,VMail Message_label0,Day Mins_label0,Day Calls_label0,Day Charge_label0,...,Eve Mins_label1,Eve Calls_label1,Eve Charge_label1,Night Mins_label1,Night Calls_label1,Night Charge_label1,Intl Mins_label1,Intl Calls_label1,Intl Charge_label1,CustServ Calls_label1
1,-0.08610271903323313,-0.026144739139116635,-0.39879154078549806,-0.30083499880355646,7.071969730197268e-16,0.3752421289682393,0.3987915407854986,0.26724346013587774,-0.16314199395770393,-0.046657497361527724,...,-0.30060422960725086,0.0010753871586025804,-0.060422960725076226,0.06404092248382845,0.30060422960725064,-0.017164454021662834,0.1555891238670701,0.0639863316296092,-1.5543122344752192e-15,0.07960557937622004
2,0.0,-0.02189378659824909,0.0,0.004346612249554738,0.0,-0.03577972162511935,0.0,0.06946638486935056,0.0,0.005107320068015199,...,0.0,-0.03684338057822025,0.0,0.06813514067265414,0.0,0.05677931755781172,0.0,-0.021911671580509696,0.0,-0.266086107678307
3,0.30305024934977165,0.07918526974349642,0.350045024660716,0.13400165478686207,0.1783410812318022,0.06542512418370894,-0.02423560924993928,0.014270462820364775,-0.10781463552879962,-0.024511163399472104,...,0.7159126750133713,0.2601034952526819,0.0077090447001180945,-0.03934757536620058,-0.2102857351818584,-0.08310748083023059,-0.16329095987091347,-0.046383135759408466,-0.8198190785714697,-0.42591944539626764
4,-0.02499999999999994,-0.0009604788385331325,-5.551115123125783e-17,0.0,1.1102230246251565e-16,5.551115123125783e-17,0.050000000000000225,0.020083975605666694,-0.1249999999999999,-0.03997107176110147,...,-0.14999999999999977,0.005332807265222163,0.30000000000000004,-0.007037025317549861,0.0,0.0,-0.6000000000000001,-0.04101655557751642,-0.3999999999999999,-0.34214822277426715


## Step 4 - Batch inference on Test Set
This is likely to be a separate scoring script for prod purposes.

You could create a batch transform through the AWS Console UI, however there are a couple of default behaviours that you can't override in the UI that are undesirable for our purposes

We want our predictions to be probability scores not class labels and we want to associate the scores with a cutomer id at the very least, as well as the original input data


Tasks are
* define the predictive columns we need (probability scores not output by default) 
* join the scores back to the input data (this is not default behaviour

In [52]:
# heres another way of getting the best candidate model name without having to expicitly specify
# the long string of characters! Instead we need to know the Jobname
automl = AutoML.attach(auto_ml_job_name)
best_candidate = automl.describe_auto_ml_job()['BestCandidate']
best_candidate_name = best_candidate['CandidateName']

In [53]:
best_candidate

{'CandidateName': 'tuning-job-1-c0d70d5fb80d43d7a2-002-bd2c97a7',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:auc',
  'Value': 0.8993300199508667},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:eu-west-1:114936231890:processing-job/db-1-0f6e6f8b86a741ec9b00a7255b30eee765437b79d4d64d319da929c99a',
   'CandidateStepName': 'db-1-0f6e6f8b86a741ec9b00a7255b30eee765437b79d4d64d319da929c99a'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:eu-west-1:114936231890:training-job/automl-chu-dpp1-1-e384a9b47d544f2aa2f3c7da0efd30ebe94bbfc74c5b4',
   'CandidateStepName': 'automl-chu-dpp1-1-e384a9b47d544f2aa2f3c7da0efd30ebe94bbfc74c5b4'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:eu-west-1:114936231890:transform-job/automl-chu-dpp1-rpb-1-72d8109d7d0b4fe8939949ffde6f589af7749a83e

In [54]:
# This is IMPORTANT- we want probability but need to explicitly specify it

inference_response_keys = ['predicted_label', 'probability']
model = automl.create_model(name=best_candidate_name,
                            candidate=best_candidate,
                            inference_response_keys=inference_response_keys)


# it should be possible to pass the name of an existing experiment in order to group
# invocations of the batch transform under the same experiment 
# however, although the doc suggests you can do this through the boto API,
#it appears this is not yet implemented as part of the python sdk yet!!

# experiment_config =  { 
#       "ExperimentName": auto_ml_job_name + "-aws-auto-ml-job",
#       "TrialComponentDisplayName": auto_ml_job_name + "aws-transform-jobv6",
#       "TrialName": "tuning-job-1-8722c107ed7946f791-003-a55c8eb6"
#    }


In [55]:
test_data_s3_path

's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/test/test_data.csv'

In [56]:
test_data_s3_path

's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/test/test_data.csv'

In [58]:
testd = pd.read_csv(test_data_s3_path)
testd.head()

,335-4719,LA,117,408,no,no.1,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1
0,379-3012,CO,83,510,no,no,0,132.4,120,22.51,121.6,101,10.34,197.7,84,8.90,8.6,2,2.32,1
1,406-5023,DC,93,408,no,no,0,164.5,95,27.97,230.9,87,19.63,149.9,91,6.75,9.9,3,2.67,4
2,333-9511,DC,141,415,no,yes,37,185.4,87,31.52,178.5,128,15.17,218.3,107,9.82,8.0,3,2.16,4
3,405-7204,ID,82,510,no,yes,34,232.6,121,39.54,153.2,115,13.02,286.7,77,12.90,4.7,3,1.27,3
4,399-9239,ID,169,415,no,no,0,235.7,79,40.07,136.9,85,11.64,220.9,97,9.94,13.3,10,3.59,1


In [59]:
output_path = s3_transform_output_path + best_candidate['CandidateName'] +'/'
transformer=model.transformer(instance_count=1, 
                          instance_type='ml.m5.xlarge',
                          assemble_with='Line',
                          accept='text/csv', 
                          output_path=output_path)


# To include the input columns in the resulting file, need to specify join source
# See: https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html
# Note this is why I elected to put the customer_id in the first columns so I can use
# the list slicing notation $[1:]  to ignore the first column

transformer.transform(job_name=transform_job_name, 
                      data=test_data_s3_path, 
                      split_type='Line', 
                      input_filter="$[1:]", 
                      join_source= "Input", 
                      output_filter="$",
                      content_type='text/csv',             
                      wait=False)


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (describe_response)
    sleep(30)
print ('transform job completed with status : ' + job_run_status)

InProgress
{'TransformJobName': 'automl-churn-model-transform-17-14-42-12', 'TransformJobArn': 'arn:aws:sagemaker:eu-west-1:114936231890:transform-job/automl-churn-model-transform-17-14-42-12', 'TransformJobStatus': 'InProgress', 'ModelName': 'tuning-job-1-c0d70d5fb80d43d7a2-002-bd2c97a7', 'TransformInput': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/test/test_data.csv'}}, 'ContentType': 'text/csv', 'CompressionType': 'None', 'SplitType': 'Line'}, 'TransformOutput': {'S3OutputPath': 's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/inference-results/tuning-job-1-c0d70d5fb80d43d7a2-002-bd2c97a7/', 'Accept': 'text/csv', 'AssembleWith': 'Line', 'KmsKeyId': ''}, 'TransformResources': {'InstanceType': 'ml.m5.xlarge', 'InstanceCount': 1}, 'CreationTime': datetime.datetime(2021, 2, 17, 15, 48, 40, 353000, tzinfo=tzlocal()), 'DataProcessing': {'InputFilter': '$[1:]', 'OutputF

Lets look at the file that was generated

In [60]:
import json
import io
from urllib.parse import urlparse
test_file = 'test_data.csv';

def get_csv_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:].strip('/')
    s3 = boto3.resource('s3')
    obj = s3.Object(bucket_name, '{}/{}'.format(prefix, file_name))
    return obj.get()["Body"].read().decode('utf-8')    
pred_csv = get_csv_from_s3(transformer.output_path, '{}.out'.format(test_file))
data_auc=pd.read_csv(io.StringIO(pred_csv))


The above is how it appears in the official example - My way, below, is a bit easier for me

In [70]:
transformer.output_path

's3://sagemaker-eu-west-1-114936231890/sagemaker/automl-churn17-14-41-35/inference-results/tuning-job-1-c0d70d5fb80d43d7a2-002-bd2c97a7/'

In [74]:
inference_file_path = transformer.output_path+'test_data.csv.out'
data_auc=pd.read_csv(inference_file_path, header=None)


In [75]:
data_auc.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,335-4719,LA,117,408,no,no,0,184.5,97,31.37,...,29.89,215.8,90,9.71,8.7,4,2.35,1,0,0.329426
1,379-3012,CO,83,510,no,no,0,132.4,120,22.51,...,10.34,197.7,84,8.90,8.6,2,2.32,1,0,0.298420
2,406-5023,DC,93,408,no,no,0,164.5,95,27.97,...,19.63,149.9,91,6.75,9.9,3,2.67,4,1,0.684078
3,333-9511,DC,141,415,no,yes,37,185.4,87,31.52,...,15.17,218.3,107,9.82,8.0,3,2.16,4,1,0.691144
4,405-7204,ID,82,510,no,yes,34,232.6,121,39.54,...,13.02,286.7,77,12.90,4.7,3,1.27,3,0,0.436432


# Step 5 - Post processing
Typically the probability scores are bucketed into quintiles for more practical usage

In [67]:
data_auc['decile'] = pd.qcut(data_auc.iloc[:,21], 10, labels=False, duplicates="drop")

In [68]:
data_auc.head()

,335-4719,LA,117,408,no,no.1,0,184.5,97,31.37,...,215.8,90,9.71,8.7,4,2.35,1,0.1,0.32942643761634827,decile
0,379-3012,CO,83,510,no,no,0,132.4,120,22.51,...,197.7,84,8.90,8.6,2,2.32,1,0,0.298420,5
1,406-5023,DC,93,408,no,no,0,164.5,95,27.97,...,149.9,91,6.75,9.9,3,2.67,4,1,0.684078,7
2,333-9511,DC,141,415,no,yes,37,185.4,87,31.52,...,218.3,107,9.82,8.0,3,2.16,4,1,0.691144,7
3,405-7204,ID,82,510,no,yes,34,232.6,121,39.54,...,286.7,77,12.90,4.7,3,1.27,3,0,0.436432,6
4,399-9239,ID,169,415,no,no,0,235.7,79,40.07,...,220.9,97,9.94,13.3,10,3.59,1,0,0.478741,6
